# Defining urban census tracts

This script identifies tracts that are urbanized and within a core-based statistical area (as of 2020). It uses data including Census TIGER/Line boundaries for census tracts, urban areas, and core-based statistical areas, as well locale classifications from the National Center for Educational Statistics.

In [ ]:
import arcgis
import arcpy
import os

In [ ]:
#set up workspaces and check database
default_gdb = arcpy.mp.ArcGISProject("CURRENT").defaultGeodatabase
arcpy.env.overwriteOutput = True
print(default_gdb)

### Identifying urban tracts on 2020 boundaries
This script draws on three boundary line data sources. Download them and import to your geodatabase before proceeding.

1) National Historic GIS 2020 census tract boundaries (https://www.nhgis.org/)

2) Census TIGER/Line files for 2020 Urban Areas (https://www2.census.gov/geo/tiger/TGRGDB20/tlgdb_2020_a_us_nationgeo.gdb.zip)

3) National Center for Education Statistics 2020 Locale Classifications (https://nces.ed.gov/programs/edge/data/edge_locale20_nces_all_us.zip)


In [ ]:
#defining inputs
tracts = default_gdb + "\\US_tracts_2020"  ##for data using 2010 tract boundaries, change to 2010 tracts
cbsa = default_gdb + "\\CBSA_2020"
urbanareas = default_gdb + "\\urbanareas_2020"
locales = default_gdb + "\\NCES_locales2020"

In [ ]:
#select relevant tracts
##tracts in urban areas 
arcpy.management.SelectLayerByLocation(
    in_layer=tracts,
    overlap_type="HAVE_THEIR_CENTER_IN",
    select_features=urbanareas,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

##and in a CBSA
arcpy.management.SelectLayerByLocation(
    in_layer=tracts",
    overlap_type="HAVE_THEIR_CENTER_IN",
    select_features=cbsa,
    search_distance=None,
    selection_type="SUBSET_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

## and not in Puerto Ricos
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=tracts,
    selection_type="SUBSET_SELECTION",
    where_clause="STATEFP <> '72'",
    invert_where_clause=None
)

In [ ]:
#join tracts with CBSA
arcpy.analysis.SpatialJoin(
    target_features=tracts,
    join_features=cbsa,
    out_feature_class=default_gdb + "\\tracts_byCBSA",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='GISJOIN "GISJOIN" true true false 14 Text 0 0,First,#,US_tract_2020,GISJOIN,0,13;GEOID "GEOID" true true false 11 Text 0 0,First,#,US_tract_2020,GEOID,0,10;STATEFP "STATEFP" true true false 2 Text 0 0,First,#,US_tract_2020,STATEFP,0,1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,US_tract_2020,COUNTYFP,0,2;TRACTCE "TRACTCE" true true false 6 Text 0 0,First,#,US_tract_2020,TRACTCE,0,5;CBSAFP "CBSAFP" true true false 5 Text 0 0,First,#,CBSA_2020,CBSAFP,0,4;CBSA_NAME "CBSA_NAME" true true false 100 Text 0 0,First,#,CBSA_2020,CBSA_NAME,0,99;CBSA_NAMELSAD "CBSA_NAMELSAD" true true false 100 Text 0 0,First,#,CBSA_2020,CBSA_NAMELSAD,0,99',
    match_option="HAVE_THEIR_CENTER_IN",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

In [ ]:
#join tracts with urban areas
arcpy.analysis.SpatialJoin(
    target_features="tracts_byCBSA",
    join_features=urbanareas,
    out_feature_class=default_gdb + "\\tracts_byCBSAUA",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='GISJOIN "GISJOIN" true true false 14 Text 0 0,First,#,tracts2020_byCBSA,GISJOIN,0,13;GEOID "GEOID" true true false 11 Text 0 0,First,#,tracts2020_byCBSA,GEOID,0,10;STATEFP "STATEFP" true true false 2 Text 0 0,First,#,tracts2020_byCBSA,STATEFP,0,1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,tracts2020_byCBSA,COUNTYFP,0,2;TRACTCE "TRACTCE" true true false 6 Text 0 0,First,#,tracts2020_byCBSA,TRACTCE,0,5;CBSAFP "CBSAFP" true true false 5 Text 0 0,First,#,tracts2020_byCBSA,CBSAFP,0,4;CBSA_NAME "NAME" true true false 100 Text 0 0,First,#,tracts2020_byCBSA,CBSA_NAME,0,99;CBSA_NAMELSAD "NAMELSAD" true true false 100 Text 0 0,First,#,tracts2020_byCBSA,CBSA_NAMELSAD,0,99;UAFP "UAFP" true true false 5 Text 0 0,First,#,urbanareas_2020,GEOID20,0,4;UA_NAME "UA_NAME" true true false 100 Text 0 0,First,#,urbanareas_2020,NAME20,0,99;UA_NAMELSAD "UA_NAMELSAD" true true false 100 Text 0 0,First,#,urbanareas_2020,NAMELSAD20,0,99',
    match_option="HAVE_THEIR_CENTER_IN",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

In [ ]:
#join tracts with locales
arcpy.analysis.SpatialJoin(
    target_features="tracts_byCBSAUA",
    join_features=locales,
    out_feature_class=default_gdb + "\\metrotracts_2020",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='GISJOIN "GISJOIN" true true false 14 Text 0 0,First,#,tracts2020_byCBSAUA,GISJOIN,0,13;GEOID "GEOID" true true false 11 Text 0 0,First,#,tracts2020_byCBSAUA,GEOID,0,10;STATEFP "STATEFP" true true false 2 Text 0 0,First,#,tracts2020_byCBSAUA,STATEFP,0,1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,tracts2020_byCBSAUA,COUNTYFP,0,2;TRACTCE "TRACTCE" true true false 6 Text 0 0,First,#,tracts2020_byCBSAUA,TRACTCE,0,5;CBSAFP "CBSAFP" true true false 5 Text 0 0,First,#,tracts2020_byCBSAUA,CBSAFP,0,4;CBSA_NAME "CBSA_NAME" true true false 100 Text 0 0,First,#,tracts2020_byCBSAUA,CBSA_NAME,0,99;CBSA_NAMELSAD "CBSA_NAMELSAD" true true false 100 Text 0 0,First,#,tracts2020_byCBSAUA,CBSA_NAMELSAD,0,99;UAFP "UAFP" true true false 5 Text 0 0,First,#,tracts2020_byCBSAUA,UAFP,0,4;UA_NAME "UA_NAME" true true false 100 Text 0 0,First,#,tracts2020_byCBSAUA,UA_NAME,0,99;UA_NAMELSAD "UA_NAMELSAD" true true false 100 Text 0 0,First,#,tracts2020_byCBSAUA,UA_NAMELSAD,0,99;LOCALE "LOCALE" true true false 2 Text 0 0,First,#,NCES_locales2020,LOCALE,0,1;LocaleType "LocaleType" true true false 255 Text 0 0,First,#,NCES_locales2020,LocaleType,0,254',
    match_option="HAVE_THEIR_CENTER_IN",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

In [ ]:
#fix tracts that are missing locale data
##select tracts with missing data
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="metrotracts_2020",
    selection_type="NEW_SELECTION",
    where_clause="LocaleType IS NULL",
    invert_where_clause=None
)

##join them to locales using a wider spatial relationship
arcpy.analysis.SpatialJoin(
    target_features="metrotracts_2020",
    join_features=locales,
    out_feature_class=default_gdb + "\\metrotracts_2020_corrected",
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping='GISJOIN "GISJOIN" true true false 14 Text 0 0,First,#,metrotracts_2020,GISJOIN,0,13;GEOID "GEOID" true true false 11 Text 0 0,First,#,metrotracts_2020,GEOID,0,10;STATEFP "STATEFP" true true false 2 Text 0 0,First,#,metrotracts_2020,STATEFP,0,1;COUNTYFP "COUNTYFP" true true false 3 Text 0 0,First,#,metrotracts_2020,COUNTYFP,0,2;TRACTCE "TRACTCE" true true false 6 Text 0 0,First,#,metrotracts_2020,TRACTCE,0,5;CBSAFP "CBSAFP" true true false 5 Text 0 0,First,#,metrotracts_2020,CBSAFP,0,4;CBSA_NAME "CBSA_NAME" true true false 100 Text 0 0,First,#,metrotracts_2020,CBSA_NAME,0,99;CBSA_NAMELSAD "CBSA_NAMELSAD" true true false 100 Text 0 0,First,#,metrotracts_2020,CBSA_NAMELSAD,0,99;UAFP "UAFP" true true false 5 Text 0 0,First,#,metrotracts_2020,UAFP,0,4;UA_NAME "UA_NAME" true true false 100 Text 0 0,First,#,metrotracts_2020,UA_NAME,0,99;UA_NAMELSAD "UA_NAMELSAD" true true false 100 Text 0 0,First,#,metrotracts_2020,UA_NAMELSAD,0,99;LOCALE "LOCALE" true true false 2 Text 0 0,First,#,NCES_locales2020,LOCALE,0,1;LocaleType "LocaleType" true true false 255 Text 0 0,First,#,NCES_locales2020,LocaleType,0,254',
    match_option="LARGEST_OVERLAP",
    search_radius=None,
    distance_field_name="",
    match_fields=None
)

##delete selected rows with missing data
arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="metrotracts_2020",
    selection_type="NEW_SELECTION",
    where_clause="LocaleType IS NULL",
    invert_where_clause=None
)
arcpy.management.DeleteRows(
    in_rows="metrotracts_2020"
)

##append corrected rows back into main layer
arcpy.management.Append(
    inputs="metrotracts_2020_corrected",
    target="metrotracts_2020",
    schema_type="TEST",
    field_mapping=None,
    subtype="",
    expression="",
    match_fields=None,
    update_geometry="NOT_UPDATE_GEOMETRY"
)